In [1]:
import tensorflow as tf
cnn_model = tf.keras.models.load_model(r"E:\CNN_model")


In [2]:
import chardet
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

with open(r"C:\Deep-Learning-based-Authorship-Identification\data_with_genre.csv", 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']

data = pd.read_csv(r"C:\Deep-Learning-based-Authorship-Identification\data_with_genre.csv", encoding=encoding)
text = list(data['text'].values)
author = list(data['author'].values)

normed_text = []
for excerpt in text:
    excerpt = excerpt.replace('\xa0', '')
    excerpt = excerpt.lower().replace("'", " ")
    excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))
    excerpt = excerpt.replace('“', '').replace('”', '')
    normed_text.append(excerpt)

tokens = [word_tokenize(sentence) for sentence in normed_text]
flat_tokens = [token for sublist in tokens for token in sublist]

unique_words = set(flat_tokens)
vocab = {word: idx for idx, word in enumerate(sorted(unique_words))}

train_data, test_data, author_train, author_test = train_test_split(normed_text, author, test_size=0.2, random_state=5)
train_tokens = [word_tokenize(sentence) for sentence in train_data]
train_tokens = [element for sublist in train_tokens for element in sublist]
test_tokens = [word_tokenize(sentence) for sentence in test_data]

In [3]:
text = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.

Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and human race form transient incidents. They have hinted at strange survivals in terms which would freeze the blood if not masked by a bland optimism. But it is not from them that there came the single glimpse of forbidden eons which chills me when I think of it and maddens me when I dream of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together of separated things—in this case an old newspaper item and the notes of a dead professor. I hope that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part he knew, and that he would have destroyed his notes had not sudden death seized him.

My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intensified by the obscurity of the cause of death. The professor had been stricken whilst returning from the Newport boat; falling suddenly, as witnesses said, after having been jostled by a nautical-looking negro who had come from one of the queer dark courts on the precipitous hillside which formed a short cut from the waterfront to the deceased's home in Williams Street. Physicians were unable to find any visible disorder, but concluded after perplexed debate that some obscure lesion of the heart, induced by the brisk ascent of so steep a hill by so elderly a man, was responsible for the end. At the time I saw no reason to dissent from this dictum, but latterly I am inclined to wonder—and more than wonder.

"""

In [4]:
from collections import Counter
def get_vocab_size(sentence_list, n, seq_size):

    token_vocab = []

    for sentence in sentence_list:
        n_grams = [sentence[i:i+n] for i in range(len(sentence) - n + 1)]
        combined_three_grams = [" ".join(sublist) for sublist in n_grams]
        new_string = ["".join(word.split()) for word in combined_three_grams]
        token_vocab.append(new_string)
    flattened_list = [item for sublist in token_vocab for item in sublist]

    n_gram_cnt = Counter(flattened_list)
    vocab_size = len(n_gram_cnt)


    return vocab_size

In [5]:
from keras.preprocessing.text import one_hot
import numpy as np 
from nltk.tokenize import word_tokenize

In [6]:
excerpt =  """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.

Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and human race form transient incidents. They have hinted at strange survivals in terms which would freeze the blood if not masked by a bland optimism. But it is not from them that there came the single glimpse of forbidden eons which chills me when I think of it and maddens me when I dream of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together of separated things—in this case an old newspaper item and the notes of a dead professor. I hope that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part he knew, and that he would have destroyed his notes had not sudden death seized him.

My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intensified by the obscurity of the cause of death. The professor had been stricken whilst returning from the Newport boat; falling suddenly, as witnesses said, after having been jostled by a nautical-looking negro who had come from one of the queer dark courts on the precipitous hillside which formed a short cut from the waterfront to the deceased's home in Williams Street. Physicians were unable to find any visible disorder, but concluded after perplexed debate that some obscure lesion of the heart, induced by the brisk ascent of so steep a hill by so elderly a man, was responsible for the end. At the time I saw no reason to dissent from this dictum, but latterly I am inclined to wonder—and more than wonder.

"""
excerpt = excerpt.replace('\xa0', '')
excerpt = excerpt.lower().replace("'", " ")
excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))
excerpt = excerpt.replace('“', '').replace('”', '')
final = word_tokenize(excerpt)
# final = [list(col) for col in zip(*final)]
n = 2
cnn_vocab_size = get_vocab_size(train_tokens, 2, 300)
seq_size = 300
n_grams = [final[i:i+n] for i in range(len(final) - n + 1)]
combined_n_grams = [" ".join(sublist) for sublist in n_grams]
new_string = ["".join(word.split()) for word in combined_n_grams]
new_string = " ".join(new_string)
hot = one_hot(new_string, round(cnn_vocab_size*1.3))

hot_len = len(hot)
print(hot_len)
if hot_len >= seq_size:
    hot = hot[0:seq_size]
else:
    diff = seq_size - hot_len
    extra = [0]*diff
    hot = hot + extra

final = np.array(hot)
final = final.reshape(1, 300)

461


In [7]:
from sklearn.preprocessing import LabelBinarizer
author_lb = LabelBinarizer()
author_lb.fit(author_train)
author_train_hot = author_lb.transform(author_train)
author_test_hot = author_lb.transform(author_test)
author_pred1 = cnn_model.predict([final, final, final, final])

author_pred1 = author_lb.inverse_transform(author_pred1)
author_pred1

1/1 [==============================] - 0s 176ms/step


array(['H.P. Lovecraft'], dtype='<U23')

In [8]:
def cnn_text(excerpt, model, vocab_size, n):
    excerpt = excerpt.replace('\xa0', '')
    excerpt = excerpt.lower().replace("'", " ")
    excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))
    excerpt = excerpt.replace('“', '').replace('”', '')
    final = word_tokenize(excerpt)
    # vocab_size = get_vocab_size(train_tokens, 1, 300)
    seq_size = 300
    n_grams = [final[i:i+n] for i in range(len(final) - n + 1)]
    combined_n_grams = [" ".join(sublist) for sublist in n_grams]
    new_string = ["".join(word.split()) for word in combined_n_grams]
    new_string = " ".join(new_string)
    hot = one_hot(new_string, round(vocab_size*1.3))

    hot_len = len(hot)
    print(hot_len)
    if hot_len >= seq_size:
        hot = hot[0:seq_size]
    else:
        diff = seq_size - hot_len
        extra = [0]*diff
        hot = hot + extra

    final = np.array(hot)
    final = final.reshape(1, 300)
    output = model.predict([final, final, final, final])
    return output

In [9]:
excerpt =  """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.

Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and human race form transient incidents. They have hinted at strange survivals in terms which would freeze the blood if not masked by a bland optimism. But it is not from them that there came the single glimpse of forbidden eons which chills me when I think of it and maddens me when I dream of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together of separated things—in this case an old newspaper item and the notes of a dead professor. I hope that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part he knew, and that he would have destroyed his notes had not sudden death seized him.

My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intensified by the obscurity of the cause of death. The professor had been stricken whilst returning from the Newport boat; falling suddenly, as witnesses said, after having been jostled by a nautical-looking negro who had come from one of the queer dark courts on the precipitous hillside which formed a short cut from the waterfront to the deceased's home in Williams Street. Physicians were unable to find any visible disorder, but concluded after perplexed debate that some obscure lesion of the heart, induced by the brisk ascent of so steep a hill by so elderly a man, was responsible for the end. At the time I saw no reason to dissent from this dictum, but latterly I am inclined to wonder—and more than wonder.

"""

In [11]:
    excerpt = excerpt.replace('\xa0', '')
    excerpt = excerpt.lower().replace("'", " ")
    excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))
    excerpt = excerpt.replace('“', '').replace('”', '')
    final = word_tokenize(excerpt)
    vocab_size = get_vocab_size(train_tokens, 1, 300)
    seq_size = 300
    n_grams = [final[i:i+n] for i in range(len(final) - n + 1)]
    combined_n_grams = [" ".join(sublist) for sublist in n_grams]
    new_string = ["".join(word.split()) for word in combined_n_grams]
    new_string = " ".join(new_string)
    hot = one_hot(new_string, round(vocab_size*1.3))

    hot_len = len(hot)
    print(hot_len)
    if hot_len >= seq_size:
        hot = hot[0:seq_size]
    else:
        diff = seq_size - hot_len
        extra = [0]*diff
        hot = hot + extra

    final = np.array(hot)
    final = final.reshape(1, 300)

461


In [12]:

output = cnn_text(excerpt,cnn_model,cnn_vocab_size,1)
output

462
1/1 [==============================] - 0s 34ms/step


array([[1.7379485e-03, 1.9670357e-03, 6.7999290e-04, 5.7598609e-03,
        1.7763715e-02, 3.3103758e-01, 4.3107984e-03, 1.2583649e-04,
        4.4795610e-02, 2.7598871e-04, 6.7906203e-03, 2.9990147e-06,
        4.6101522e-02, 3.5091561e-01, 1.4722964e-03, 4.0910927e-05,
        7.2039571e-04, 1.4427445e-04, 5.3295102e-05, 8.0072597e-02,
        6.8280019e-02, 3.3724364e-02, 3.2267210e-03]], dtype=float32)

In [13]:
import torch.nn as nn
class AuthorClassifier(nn.Module):
    def __init__(self, mode, output_size, hidden_size, vocab_size, embedding_length, embeddings):
        super(AuthorClassifier, self).__init__()
        if mode not in ['rnn', 'lstm', 'gru', 'bilstm']:
            raise ValueError("Choose a mode from - rnn / lstm / gru / bilstm")

        self.mode = mode
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.embedding = nn.Embedding.from_pretrained(embeddings, freeze=True)

        if self.mode == 'rnn':
            self.network = nn.RNN(self.embedding_length, self.hidden_size, batch_first=True)
        elif self.mode == 'lstm':
            self.network = nn.LSTM(self.embedding_length, self.hidden_size, batch_first=True)
        elif self.mode == 'gru':
            self.network = nn.GRU(self.embedding_length, self.hidden_size, batch_first=True)
        elif self.mode == 'bilstm':
            self.network = nn.LSTM(self.embedding_length, self.hidden_size, bidirectional=True, batch_first=True)

        self.linear = nn.Linear(self.hidden_size * 2 if self.mode == 'bilstm' else self.hidden_size, self.output_size)

    def forward(self, text, text_lengths):
        text_embeddings = self.embedding(text)
        # Ensure lengths are on CPU and sorted if not already handled outside
        packed_input = nn.utils.rnn.pack_padded_sequence(text_embeddings, text_lengths.cpu(), batch_first=True, enforce_sorted=False)

        if self.mode in ('lstm', 'bilstm'):
            _, (hidden, _) = self.network(packed_input)
        else:
            _, hidden = self.network(packed_input)

        if self.mode == 'bilstm':  # Concatenate the hidden states from both directions
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]
        hidden = hidden.squeeze(0)
        output = self.linear(hidden)
        return output

In [14]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator, GloVe
from torchtext.data.utils import get_tokenizer


train_df = pd.read_csv(r'C:\Deep-Learning-based-Authorship-Identification\project\my_train_7030_1.csv')
test_df = pd.read_csv(r'C:\Deep-Learning-based-Authorship-Identification\project\my_test_7030_1.csv')
def yield_tokens(data_iter):
    tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
    for text in data_iter:
        yield tokenizer(text)

# Assuming train_df and test_df are your dataframes with text data
all_texts = list(train_df['text'].str.lower()) + list(test_df['text'].str.lower())
# Build vocabulary
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
vocab = build_vocab_from_iterator(yield_tokens(all_texts), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab['<unk>'])  # Handle unknown tokens

# Load GloVe embeddings
glove = GloVe(name='6B', dim=100)
# Prepare the embedding matrix
embedding_matrix = torch.zeros((len(vocab), 100))
for word, idx in vocab.get_stoi().items():
    if word in glove.stoi:
        embedding_matrix[idx] = glove.vectors[glove.stoi[word]]
    else:
        embedding_matrix[idx] = torch.randn(100)  # Random initialization for words not in GloVe

In [17]:
# Initialize the model
import torch
mode = 'lstm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_size = 23
hidden_size = 300
vocab_size = len(vocab)
embedding_length = 100
word_embeddings = nn.Embedding(vocab_size, embedding_length)
lstm_model = AuthorClassifier(mode = mode, output_size = output_size,hidden_size=hidden_size,vocab_size=len(vocab),embedding_length=embedding_length,embeddings= embedding_matrix)


# Load the saved state dictionary
lstm_model.load_state_dict(torch.load(r'C:\Users\marti\Downloads\lstm.pth'))

# Optionally, switch to evaluation mode



<All keys matched successfully>

In [18]:
from torchtext.data.utils import get_tokenizer
def predict_single_text(text, model, vocab, device):
    model.eval()  # Set the model to evaluation mode
    model.to(device)

    # Tokenize the input text
    tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
    tokens = tokenizer(text.lower())
    indices = [vocab[token] if token in vocab else vocab['<unk>'] for token in tokens]

    # Ensure indices are in the correct type (torch.LongTensor)
    indices_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)

    # Length of input sequence
    lengths = torch.tensor([len(indices)], dtype=torch.long).to(device)

    with torch.no_grad():
        # Forward pass, ensure that the model and input are compatible
        outputs = model(indices_tensor, lengths)
        
        # Check output dimension and adjust if necessary
        if outputs.dim() == 1:
            outputs = outputs.unsqueeze(0)

        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_idx = torch.argmax(probabilities, dim=1).item()

    # Return all class probabilities and the predicted class index
    return  predicted_idx , probabilities.squeeze().tolist()

In [19]:
a,b = predict_single_text(excerpt,lstm_model,vocab,'cpu')

In [20]:
import torch
mode = 'gru'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_size = 23
hidden_size = 300
vocab_size = len(vocab)
embedding_length = 100
word_embeddings = nn.Embedding(vocab_size, embedding_length)
gru_model = AuthorClassifier(mode = mode, output_size = output_size,hidden_size=hidden_size,vocab_size=len(vocab),embedding_length=embedding_length,embeddings= embedding_matrix)


# Load the saved state dictionary
gru_model.load_state_dict(torch.load(r'C:\Users\marti\Downloads\gru.pth'))

<All keys matched successfully>

In [21]:
import torch
mode = 'bilstm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_size = 23
hidden_size = 400
vocab_size = len(vocab)
embedding_length = 100
word_embeddings = nn.Embedding(vocab_size, embedding_length)
bilstm_model = AuthorClassifier(mode = mode, output_size = output_size,hidden_size=hidden_size,vocab_size=len(vocab),embedding_length=embedding_length,embeddings= embedding_matrix)


# Load the saved state dictionary
bilstm_model.load_state_dict(torch.load(r'C:\Users\marti\Downloads\bilstm.pth'))

<All keys matched successfully>

-----------------------transformer model ------------------------------

In [22]:
from transformers import BertForSequenceClassification

# Initialize the model with the same configuration
transformer_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=23)
model_path = r"C:\Users\marti\Downloads\transformer_model.pth"
transformer_model.load_state_dict(torch.load(model_path))
transformer_model.eval()

c:\Users\marti\miniconda3\envs\csci2470project2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [58]:
def predict_text(text, model, tokenizer, device):
    # Encode text
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Move encoded input to correct device
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    
    # Model inference
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    # Assuming you want to get the maximum probability index
    predicted_index = torch.argmax(probabilities, dim=1).item()
    
    # predicted_prob = probabilities.squeeze()[predicted_index].item()
    predicted_prob = probabilities.squeeze().cpu().numpy()
    print(predicted_prob)

    return predicted_index, predicted_prob

# Usage
# sample_text = """
# The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.
# """
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transformer_model.to('cpu')
# predicted_index, predicted_prob = predict_text(sample_text, transformer_model, tokenizer, device)


In [59]:
device = 'cpu'

In [60]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AdaptiveDropout(Layer):
    def init(self, initial_rate=0.2):
        super(AdaptiveDropout, self).init()
        self.dropout_rate = tf.Variable(initial_rate, trainable=False)

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.dropout_rate)
        return inputs

    def set_rate(self, new_rate):
        self.dropout_rate.assign(new_rate)

class AdjustDropoutRateCallback(tf.keras.callbacks.Callback):
    def init(self, model, threshold=0.90, increase_by=0.1):
        super().init()
        for layer in model.layers:
            if isinstance(layer, AdaptiveDropout):
                layer_adap = layer
                break
        self.layer = layer_adap
        self.threshold = threshold
        self.increase_by = increase_by

    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > self.threshold:
            new_rate = min(1.0, self.layer.dropout_rate + self.increase_by)
            self.layer.set_rate(new_rate)
            print(f"Epoch {epoch+1}: Increasing dropout rate to {new_rate:.2f}")
        else:
            print(f"Epoch {epoch+1}: Current accuracy {logs['accuracy']:.2f} is below threshold")

In [61]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense, concatenate

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

 

def define_model2(input_len, output_size, vocab_size, embedding_dim, verbose=True,

                  drop_out_pct=0.25, conv_filters=500, activation_fn='leaky_relu',

                  pool_size=2, learning=0.001, init_mode_embedding=None, init_mode_conv=None, channel = 4):

    """Define n-gram CNN with different weight initializations for each channel.

   

    Args:

    input_len: int. Length of input sequences.

    output_size: int. Number of output classes.

    vocab_size: int. Maximum value of n-gram encoding.

    embedding_dim: int. Size of embedding layer.

    verbose: bool. Whether or not to print model summary.

    drop_out_pct: float. Drop-out rate.

    conv_filters: int. Number of filters in the conv layer.

    activation_fn: string. Activation function to use in the convolutional layer.

    pool_size: int. Pool size for the max pooling layer.

    learning: float. Learning rate for the model optimizer.

    init_mode_embedding: list of keras.initializers. Initializers for the embedding layers.

    init_mode_conv: list of keras.initializers. Initializers for the convolutional layers.

   

    Returns:

    model: keras model object.

    """

    if init_mode_embedding is None:

        init_mode_embedding = [None] * channel

    if init_mode_conv is None:

        init_mode_conv = [None] * channel

 

    channels = []

    inputs = []

 

    for i in range(channel):

        input_shape = Input(shape=(input_len,))

        inputs.append(input_shape)

        embedding = Embedding(vocab_size, embedding_dim, embeddings_initializer=init_mode_embedding[i])(input_shape)

        drop = AdaptiveDropout(drop_out_pct)(embedding)

        conv = Conv1D(filters=conv_filters, kernel_size=i+3, activation=activation_fn, kernel_initializer=init_mode_conv[i])(drop)

        pool = MaxPooling1D(pool_size=pool_size)(conv)

        flat = Flatten()(pool)

        channels.append(flat)

 

    # Merge channels

    merged = concatenate(channels)

   

    

    output = Dense(output_size, activation='softmax')(merged)

  

    model = Model(inputs=inputs, outputs=output)

   

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning), metrics=['accuracy'])

   

    if verbose:

        model.summary()

       

    return model

In [62]:
import numpy as np

from tensorflow.keras.utils import to_categorical

 

def train_models(X_train, y_train, num_models=3, num_channels = 4):

    models = []

    n_samples = X_train.shape[0]

    output_size = len(np.unique(y_train))

 

    for _ in range(num_models):

        indices = np.random.choice(n_samples, n_samples, replace=True)

        X_sample = X_train[indices]

        y_sample = y_train[indices]

 

        model =  define_model2(300, 23, max_1gram + 1, 16, learning=0.1, drop_out_pct=0.0001, channel=num_channels, init_mode_embedding=[RandomNormal(mean=0, stddev=1), RandomNormal(mean=5, stddev=2), RandomNormal(mean=-3, stddev=0.5), RandomNormal(mean=10, stddev=3)], init_mode_conv=[RandomNormal(mean=0, stddev=1), RandomNormal(mean=5, stddev=2), RandomNormal(mean=-3, stddev=0.5), RandomNormal(mean=10, stddev=3)])

        model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

        dropout_adjustment_callback = AdjustDropoutRateCallback(model)

        model.fit([X_sample]*num_channels, y_sample, epochs=4, batch_size=16,

                verbose = 1, validation_split = 0.2, callbacks=[dropout_adjustment_callback])

        models.append(model)

   

    return models

 

# def majority_vote(models, X_test, num_channels = 4):

 

#     predictions = [np.argmax(model.predict([X_test]*num_channels), axis=-1) for model in models]

 

#     predictions = np.array(predictions).T

 

#     final_predictions = [np.bincount(preds).argmax() for preds in predictions]

 

#     return final_predictions

 

def majority_vote(models, excerpt_gunn, num_channels = 4):


    excerpt_gunn = excerpt_gunn.replace('\xa0', '')
    excerpt_gunn = excerpt_gunn.lower().replace("'", " ")
    excerpt_gunn = excerpt_gunn.translate(str.maketrans('', '', string.punctuation))
    excerpt_gunn = excerpt_gunn.replace('“', '').replace('”', '')
    final = word_tokenize(excerpt_gunn)
    # vocab_size = get_vocab_size(train_tokens, 1, 300)
    seq_size = 300
    n_grams = [final[i:i+n] for i in range(len(final) - n + 1)]
    combined_n_grams = [" ".join(sublist) for sublist in n_grams]
    new_string = ["".join(word.split()) for word in combined_n_grams]
    new_string = " ".join(new_string)
    hot = one_hot(new_string, round(vocab_size*1.3))

    hot_len = len(hot)
    if hot_len >= seq_size:
        hot = hot[0:seq_size]
    else:
        diff = seq_size - hot_len
        extra = [0]*diff
        hot = hot + extra

    final = np.array(hot)
    final = final.reshape(1, 300)

    

    predictions = np.array([model.predict([final]*num_channels) for model in models])

 

    predictions = np.sum(predictions, axis=0)

 

    predictions = predictions/len(models)




    return predictions

In [63]:
models = []

for i in range(11):
    path = '''e:\model_3'''
    model = tf.keras.models.load_model(path + f'\model_{i}')
    models.append(model)


Exception ignored in: <function UniquePtr.__del__ at 0x0000024915FC5E50>
Traceback (most recent call last):
  File "c:\Users\marti\miniconda3\envs\csci2470project2\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 71, in __del__
    obj = self._obj
KeyboardInterrupt: 


In [ ]:
final.shape

(1, 300)

In [32]:
text


"\nThe most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.\n\nTheosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and human race form transient incidents. They have hinted at strange survivals in terms which would freeze the blood if not masked by a bland optimism. But it is not from them that there came the single glimpse of forbidden eons which chills me when I think of it and maddens me when I dream of it. That glimpse, 

In [33]:
predictions = majority_vote(models, excerpt_gunn = text)

1/1 [==============================] - 0s 124ms/step


In [34]:
predictions

array([[2.80222818e-02, 5.11070453e-02, 6.38574595e-04, 7.50308245e-05,
        7.52568711e-03, 1.16227455e-01, 1.54160395e-01, 7.31661683e-04,
        2.98211090e-02, 2.73521572e-01, 4.82296795e-02, 1.06719257e-02,
        1.14376461e-02, 2.37817355e-02, 1.83825754e-03, 4.69149649e-03,
        2.19318904e-02, 2.63051409e-03, 2.01253612e-02, 7.29980990e-02,
        1.54841403e-02, 9.08801109e-02, 1.34683279e-02]], dtype=float32)

In [35]:
whole_data = pd.read_csv(r'C:\Deep-Learning-based-Authorship-Identification\data_with_genre.csv')
whole_data 

,text,book_name,author,genre
0,Happy families are all alike; every unhappy fa...,Anna_Kerinena,Leo Tolstoy,historical
1,"""Ah, ah, ah! Oo!..."" he muttered, recalling ev...",Anna_Kerinena,Leo Tolstoy,historical
2,But he felt all the difficulty of his position...,Anna_Kerinena,Leo Tolstoy,historical
3,"Stepan Arkadyevitch's eyes asked: ""Why do you ...",Anna_Kerinena,Leo Tolstoy,historical
4,"Beg her forgiveness, sir. There's no help for ...",Anna_Kerinena,Leo Tolstoy,historical
...,...,...,...,...
13149,"He returned after midnight, and, instead of go...",Wuthering_Heights,Emily Bronte,romance
13150,“Strange happiness! If you would hear me witho...,Wuthering_Heights,Emily Bronte,romance
13151,"The following evening was very wet: indeed, it...",Wuthering_Heights,Emily Bronte,romance
13152,"Hareton, with a streaming face, dug green sods...",Wuthering_Heights,Emily Bronte,romance


------------------------------committee-----------------------------------------

In [148]:
import pandas as pd
data_with_genre = pd.read_csv('C:\Deep-Learning-based-Authorship-Identification\data_with_genre.csv')
data_with_genre = data_with_genre.drop(columns = ['book_name','genre'])
# def create_author_dict(df, author_column):
#     # Combine unique authors from both dataframes
#     combined_authors = df[author_column].unique()
#     # Sort and create a single dictionary for all authors
#     sorted_authors = sorted(combined_authors)
#     return {i: sorted_authors[i] for i in range(len(sorted_authors))}
# author_dict = create_author_dict(data_with_genre,'author')
# author_dict2 = {}
# for key, value in author_dict.items(): 
#     author_dict2[value] = key
# author_dict2

In [149]:
import numpy as np
encoded_cnn_prediction = np.load(r"C:\Users\marti\Downloads\array.npy")
encoded_cnn_prediction.shape

(13154, 23)

In [150]:
import numpy as np

# Replace 'path_to_file.csv' with the path to your CSV file
transformer_prediction = np.loadtxt(r"C:\Users\marti\Downloads\transformer_prediction.csv", delimiter=',')  # specify delimiter if necessary
transformer_prediction.shape


(13154, 23)

In [151]:
import numpy as np

# Replace 'path_to_file.csv' with the path to your CSV file
gru_prediction = np.loadtxt(r"C:\Users\marti\Downloads\gru_prediction.csv", delimiter=',')  # specify delimiter if necessary
gru_prediction.shape

(13154, 23)

In [152]:
import numpy as np

# Replace 'path_to_file.csv' with the path to your CSV file
lstm_prediction = np.loadtxt(r"C:\Users\marti\Downloads\lstm_prediction.csv", delimiter=',')  # specify delimiter if necessary
lstm_prediction.shape

(13154, 23)

In [153]:
import numpy as np

# Replace 'path_to_file.csv' with the path to your CSV file
bilstm_prediction = np.loadtxt(r"C:\Users\marti\Downloads\bilstm_prediction.csv", delimiter=',')  # specify delimiter if necessary
bilstm_prediction.shape

(13154, 23)

In [158]:
data_with_genre['author_num_1'] = data_with_genre['author'].map(reversed_dict)
data_with_genre['author_num_2'] = data_with_genre['author_num_1'].map(author_label)


In [159]:
data_with_genre

,text,author,author_num_1,author_num_2
0,Happy families are all alike; every unhappy fa...,Leo Tolstoy,13,9
1,"""Ah, ah, ah! Oo!..."" he muttered, recalling ev...",Leo Tolstoy,13,9
2,But he felt all the difficulty of his position...,Leo Tolstoy,13,9
3,"Stepan Arkadyevitch's eyes asked: ""Why do you ...",Leo Tolstoy,13,9
4,"Beg her forgiveness, sir. There's no help for ...",Leo Tolstoy,13,9
...,...,...,...,...
13149,"He returned after midnight, and, instead of go...",Emily Bronte,5,14
13150,“Strange happiness! If you would hear me witho...,Emily Bronte,5,14
13151,"The following evening was very wet: indeed, it...",Emily Bronte,5,14
13152,"Hareton, with a streaming face, dug green sods...",Emily Bronte,5,14


In [238]:
import numpy as np

import tensorflow as tf

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Flatten, Dense, Softmax

 

class comit(tf.keras.Model):

    def __init__(self, input_size,x_1):


        super(comit, self).__init__()
        
        self.x_1 = x_1

        self.flatten = Flatten()

        self.dense = Dense(302542, activation='softmax')

 

    def call(self,x_1):
        # print(x_1.shape)
        # x = self.flatten(x_1)
        # print(x.shape)
        x = self.flatten(x_1)
        print(x.shape)
        x = self.dense(x)
        
        x = tf.reshape(x, (5, -1))
        
        x = tf.reduce_sum(x, axis=0)
        
        x = tf.reshape(x, (-1,23))

       

 

        return x

   

    def flatten(flatten_list):

        final_list = []

        for list in flatten_list:

            print(list.shape)

            list.flatten()

            final_list.append(list)

        return np.array(final_list)

In [239]:
def flatten_1(flatten_list):

    final_list = []

    for list in flatten_list:

        final_list.append(list.flatten())

    return np.array(final_list)

In [240]:
flatten_prediction = flatten_1([encoded_cnn_prediction,gru_prediction,lstm_prediction,bilstm_prediction,transformer_prediction])
flatten_prediction.shape

(5, 302542)

In [241]:
encoded_cnn_prediction.flatten().shape

(302542,)

In [168]:
np.array(data_with_genre['author_num_2'])

array([ 9,  9,  9, ..., 14, 14, 14], dtype=int64)

In [268]:
import numpy as np
# def committee_grad(x_1, x_2, x_3, x_4, x_5, t):
#     a = np.random.randn(1, 23)
#     b = np.random.randn(1, 23)
#     c = np.random.randn(1, 23)
#     d = np.random.randn(1, 23)
#     e = np.random.randn(1, 23)
#     print("initial a", a)

#     loss = float("inf")
#     converge = False
#     n = x_1.shape[0]
#     alpha = 0.01
#     epsilon = 1e-4
    
#     while not converge:
#         pred = x_1 @ a.T + x_2 @ b.T + x_3 @ c.T + x_4 @ d.T + x_5 @ e.T

#         grad_a = (2 / n) * np.sum((pred - t) * x_1, axis=0, keepdims=True)
#         grad_b = (2 / n) * np.sum((pred - t) * x_2, axis=0, keepdims=True)
#         grad_c = (2 / n) * np.sum((pred - t) * x_3, axis=0, keepdims=True)
#         grad_d = (2 / n) * np.sum((pred - t) * x_4, axis=0, keepdims=True)
#         grad_e = (2 / n) * np.sum((pred - t) * x_5, axis=0, keepdims=True)

#         a -= alpha * grad_a
#         b -= alpha * grad_b
#         c -= alpha * grad_c
#         d -= alpha * grad_d
#         e -= alpha * grad_e

#         new_pred = x_1 @ a.T + x_2 @ b.T + x_3 @ c.T + x_4 @ d.T + x_5 @ e.T
#         new_loss = np.mean(np.square(new_pred - t))
#         if np.abs(loss - new_loss) < epsilon:
#             converge = True

#         loss = new_loss  
#         print(new_loss)
#     print("final a", a)

#     return a, b, c, d, e
import numpy as np

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def committee_grad(x_1, x_2, x_3, x_4, x_5, t, max_iteration, l_rate, epsilon):
    w = np.random.randn(23*5, 23)
    model = np.concatenate((x_1, x_2, x_3, x_4, x_5), axis=1)
    print(model.shape)

    converge = False
    n = x_1.shape[0]
    alpha = l_rate
    epsilon = epsilon
    i = 0
    while not converge:
        logit = model @ w
        prob = softmax(logit)
        dL_dy = t - prob
        grad = np.einsum('ki,kj->kij', prob, prob)
        diagonal = np.einsum('ki,ij->kij', prob, np.eye(prob.shape[1]))
        grad = diagonal - grad
        dL_dlogit = np.einsum('kij,kj->ki', grad, dL_dy)
        dL_dw = model.T @ dL_dlogit

        loss = -np.sum(t * np.log(prob)) / n
        print(f'Loss: {loss}')  # Reporting loss for monitoring

        w += alpha * dL_dw
        new_pred = softmax(model @ w)
        new_loss = -np.sum(t * np.log(new_pred)) / n

        if abs(new_loss - loss) < epsilon:
            converge = True
        i += 1

        if i > max_iteration:
            break

    print(f'Final loss: {new_loss}')
    return w



def committe_train(cnn_pred, gru_pred, lstm_pred, bilstm_pred, transformer_pred,train_data, max_iter, l_rate, epsilon):

    # cnn_pred = []
    # gru_pred = []
    # lstm_pred = []
    # bilstm_pred = []
    # transformer_pred = []
    # i = 0
    # for _,(para, author) in train_data.iterrows():
    #     print(i)
    #     i += 1
    #     cnn_output = cnn_text(para, cnn_model,cnn_vocab_size,1)
    #     cnn_pred.append(cnn_output)

    #     _,gru_output = predict_single_text(para, gru_model, vocab, device)
    #     gru_pred.append(gru_output)

    #     _,lstm_output = predict_single_text(para, lstm_model, vocab, device)
    #     lstm_pred.append(lstm_output)

    #     _,bilstm_output = predict_single_text(para, bilstm_model, vocab, device)
    #     bilstm_pred.append(bilstm_output)

    #     _, transformer_output = predict_text(para, transformer_model, tokenizer, device)
    #     transformer_pred.append(transformer_output)

    author = list(train_data['author'])
    author_lb = LabelBinarizer()

    author_lb.fit(author)
    author_hot = author_lb.transform(author)

    # cnn_pred = np.array(cnn_pred)
    # print('cnn_pred',cnn_pred.shape)
    # gru_pred = np.array(gru_pred)
    # print('gru_pred',gru_pred.shape)
    # lstm_pred = np.array(lstm_pred)
    # print('lstm_pred',lstm_pred.shape)
    # bilstm_pred = np.array(bilstm_pred)
    # print('bilstm_pred',bilstm_pred.shape)
    # transformer_pred = np.array(transformer_pred)
    # print('transformer_pred',transformer_pred.shape)


    w = committee_grad(cnn_pred, gru_pred, lstm_pred, bilstm_pred, transformer_pred, author_hot, max_iter, l_rate, epsilon)
    print("train a", w)
    return w


def committe_pred(cnn_model, gru_model, lstm_model, bilstm_model, transformer_model, w, data):

    cnn_pred = []
    gru_pred = []
    lstm_pred = []
    bilstm_pred = []
    transformer_pred = []
    
    for _,(para, author) in data.iterrows():
        cnn_output = majority_vote(gunn_model, para)
        cnn_output_new = np.zeros(23)
        for i in range(len(cnn_output)):
            cnn_output_new[i] = cnn_output[author_label[i]]

        cnn_pred.append(cnn_output_new)

        _,gru_output = predict_single_text(para, gru_model, vocab, device)
        gru_pred.append(gru_output)

        _,lstm_output = predict_single_text(para, lstm_model, vocab, device)
        lstm_pred.append(lstm_output)

        _,bilstm_output = predict_single_text(para, bilstm_model, vocab, device)
        bilstm_pred.append(bilstm_output)

        _, transformer_output = predict_text(para, transformer_model, tokenizer, device)
        transformer_pred.append(transformer_output)

    cnn_pred = np.array(cnn_pred)
    gru_pred = np.array(gru_pred)
    lstm_pred = np.array(lstm_pred)
    bilstm_pred = np.array(bilstm_pred)
    transformer_pred = np.array(transformer_pred)
    model = np.concatenate((cnn_pred, gru_pred, lstm_pred, bilstm_pred, transformer_pred), axis=1)

    pred = model @ w

    return pred


In [269]:
from sklearn.preprocessing import LabelBinarizer
w = committe_train(encoded_cnn_prediction, gru_prediction, lstm_prediction, bilstm_prediction, transformer_prediction,data_with_genre, 5000, 0.001, 1e-4)

(13154, 115)
Loss: 4.92180844044516
Loss: 4.845294352618584
Loss: 4.766943710654446
Loss: 4.686047735650494
Loss: 4.601945631425421
Loss: 4.514093720787734
Loss: 4.422182051412075
Loss: 4.326330320950224
Loss: 4.2273665121725035
Loss: 4.127085672107641
Loss: 4.028200328496655
Loss: 3.9336886917143086
Loss: 3.8457793461268306
Loss: 3.765354236739301
Loss: 3.692139031941517
Loss: 3.625270156840749
Loss: 3.5637560908696715
Loss: 3.5067103495241483
Loss: 3.453430869834052
Loss: 3.4034044295205486
Loss: 3.3562774001549576
Loss: 3.3118123501581556
Loss: 3.269841454570326
Loss: 3.23022472765774
Loss: 3.1928194896327833
Loss: 3.1574645905353975
Loss: 3.123978557589943
Loss: 3.0921670993503483
Loss: 3.0618341996965017
Loss: 3.0327923011860745
Loss: 3.004869351815297
Loss: 2.9779124085064823
Loss: 2.951788518762338
Loss: 2.9263838674173503
Loss: 2.901602029994535
Loss: 2.8773619053343205
Loss: 2.8535956572787846
Loss: 2.830246824679198
Loss: 2.8072686569495504
Loss: 2.7846226795125086
Loss: 2.76

In [270]:
w.shape

(115, 23)

In [282]:

def committe_pred(gunn_model, gru_model, lstm_model, bilstm_model, transformer_model, w, data):

    cnn_pred = []
    gru_pred = []
    lstm_pred = []
    bilstm_pred = []
    transformer_pred = []
    print(a)
    j = 0
    for _,(para, index, author) in data.iterrows():
        print(j)
        j+= 1
        cnn_output = majority_vote(gunn_model, para)
        cnn_output_new = np.zeros(23)
        for i in range(len(cnn_output)):
            print('cnn_output',cnn_output)
            cnn_output_new[i] = cnn_output[0][author_label[i]]

        cnn_pred.append(cnn_output_new)

        _,gru_output = predict_single_text(para, gru_model, vocab, device)
        gru_pred.append(gru_output)

        _,lstm_output = predict_single_text(para, lstm_model, vocab, device)
        lstm_pred.append(lstm_output)

        _,bilstm_output = predict_single_text(para, bilstm_model, vocab, device)
        bilstm_pred.append(bilstm_output)

        _, transformer_output = predict_text(para, transformer_model, tokenizer, device)
        transformer_pred.append(transformer_output)

    cnn_pred = np.array(cnn_pred)
    # cnn_pred = np.squeeze(cnn_pred, axis=1)
    gru_pred = np.array(gru_pred)
    lstm_pred = np.array(lstm_pred)
    bilstm_pred = np.array(bilstm_pred)
    # Assuming transformer_pred is the tensor that caused the error
    transformer_pred = np.array(transformer_pred)
    model = np.concatenate((cnn_pred, gru_pred, lstm_pred, bilstm_pred, transformer_pred), axis=1)

    print(cnn_pred.shape, gru_pred.shape, lstm_pred.shape, bilstm_pred.shape, transformer_pred)

    pred = softmax(model @ w)

    return pred

In [289]:
final_nb_output = committe_pred(models, gru_model, lstm_model, bilstm_model, transformer_model, w, data)

[[ 0.98330117  1.28791763  1.23525865  3.62597396  1.55485025  1.64584446
   1.61159307 -0.74751807  0.87858712  2.65801443  2.2786389   0.63822682
   2.21087071  3.77740929  0.46998776  2.02303151  1.9616533  -0.09932634
   1.81661745  1.07324096  0.67641063  2.23210791  1.33063725]]
0
1/1 [==============================] - 0s 166ms/step
cnn_output [[7.2785229e-03 1.5618852e-02 1.6906604e-03 6.7428817e-05 2.3305310e-02
  8.5372701e-02 2.7590623e-01 1.8481347e-03 3.6685392e-02 2.3434004e-01
  2.1031419e-02 5.6422432e-03 7.2886201e-04 2.9735027e-02 8.1150169e-03
  1.3165146e-03 2.4462648e-02 1.3657316e-03 8.9163393e-02 2.9933276e-02
  1.0267832e-02 7.1088314e-02 2.5036430e-02]]
[1.04910405e-05 9.96407092e-01 1.25793784e-04 1.16460003e-04
 2.24851057e-04 4.76795431e-05 6.42021711e-04 7.17527582e-05
 1.33246271e-04 2.75707222e-04 1.20679710e-04 4.80651433e-05
 6.04923407e-05 8.10142592e-05 6.74591138e-05 1.52862194e-04
 2.89098854e-04 2.75223192e-05 4.23815982e-05 3.45170381e-04
 2.819072

In [290]:
label = np.argmax(final_nb_output)


In [291]:
final_nb_output.shape

(690, 23)

In [292]:
nb_pred_index = np.argmax(final_nb_output, axis =1)
nb_pred_index

array([21, 18,  7,  4, 18,  3,  3,  2,  1,  1, 20,  8,  7, 15, 14, 22,  6,
       21,  9, 15, 13,  5,  7,  4,  3, 21, 11,  8, 15,  3, 11, 18,  5,  5,
       14,  7,  9, 18,  3, 16,  9,  1, 17, 19,  6,  6, 14,  8,  8, 16, 16,
       18,  7, 21, 14, 16, 18,  9, 12, 20,  8, 18,  2,  6,  5, 10, 20, 13,
        9, 21, 16, 11,  3,  0,  4,  6, 13,  2, 16, 15,  8, 22, 17, 18, 17,
       13,  4,  8,  5, 19, 11,  2, 21, 19,  9,  3,  0,  2,  0,  3,  0, 20,
        5,  4, 17,  0, 15,  0,  9,  1, 21,  8, 17, 15,  1, 11, 16, 16,  0,
       13,  4, 18,  8, 11, 11,  4,  3,  2,  8, 17,  1, 21, 20,  9,  8, 14,
        8, 15, 16,  2, 21, 19,  5, 11, 18, 22,  7, 13,  7,  7, 21, 18, 14,
        9,  3,  1,  6, 17, 15, 16, 16, 14,  9,  6,  1,  4, 21, 21,  7, 17,
        0,  7,  0, 16, 14, 21,  1,  9, 11,  8, 16,  0, 14,  4,  3,  7, 20,
       15, 18,  1, 10, 16,  1, 21, 20, 17,  4, 20,  8,  4, 19, 18,  6,  4,
       10,  8, 10,  0,  3, 10,  7, 13, 19, 21, 16,  7,  4,  7,  3, 15, 11,
       18, 17,  1, 18, 10

Test accuracy of Committee

In [294]:
from sklearn.metrics import accuracy_score
reversed_dict = {
    'Agatha Christie': 0,
    'Arthur Conan Doyle': 1,
    'Austen': 2,
    'Daniel Defoe': 3,
    'Dickens': 4,
    'Emily Bronte': 5,
    'Fitzgerald': 6,
    'Frances Hodgson Burnett': 7,
    'Gaston Leroux': 8,
    'H.P. Lovecraft': 9,
    'Herman Melville': 10,
    'J.K. Rowling': 11,
    'Jules Verne': 12,
    'Leo Tolstoy': 13,
    'Louisa May Alcott': 14,
    'Mark Twain': 15,
    'Mitchell': 16,
    'Montgomery': 17,
    'Robert Louis Stevenson': 18,
    'Shelley': 19,
    'Stoker': 20,
    'Victor Hugo': 21,
    'Wilkie Collins': 22
}
test_data_snippet = data
test_data_snippet['author_num'] = test_data_snippet['author'].map(reversed_dict)
test_data_snippet
accuracy = accuracy_score(data['author'].map(reversed_dict), nb_pred_index)
accuracy


0.9043478260869565

In [88]:
nb_pred_index

array([9, 9], dtype=int64)

In [ ]:
import os
import tensorflow as tf

# Path to the directory containing the model files
directory_path = "E:\model_3"

# List all files in the directory and filter for model files (e.g., .h5 files)
model_files = [file for file in os.listdir(directory_path) if file.endswith('.h5')]

# List to store the loaded models
loaded_models = []

# Loop through the model files and load each one
for model_file in model_files:
    full_path = os.path.join(directory_path, model_file)
    model = tf.keras.models.load_model(full_path)
    loaded_models.append(model)
    print(f"Loaded model from {full_path}")

# 'loaded_models' now contains all the loaded TensorFlow models


In [110]:
i= 0 
gru_pred = []
for _,(para, boo_name,author,author_num) in data.iterrows():
        i+= 1
        print(i)
        _,gru_output = predict_single_text(para, gru_model, vocab, device)
        gru_pred.append(gru_output)
gru_pred

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[[1.657812208577525e-05,
  0.14455477893352509,
  5.727962161472533e-06,
  7.75751018977644e-10,
  6.782039463359979e-07,
  1.613486944052056e-08,
  2.5789828050903907e-09,
  1.3885781413591758e-07,
  1.4337072640202564e-09,
  2.057581468761782e-06,
  1.4511248869197857e-09,
  1.333851962215249e-08,
  1.1152719991969207e-07,
  6.638186732743634e-06,
  2.0141314962529577e-06,
  4.657943645725027e-05,
  0.854204535484314,
  8.193614675633398e-09,
  1.9638859427573152e-08,
  2.511345265077125e-08,
  3.2915415886236588e-06,
  1.6465519365738146e-06,
  0.0011553914519026875],
 [3.210163049516268e-05,
  1.1178697150171502e-06,
  6.803223300266836e-07,
  5.368809752326342e-08,
  0.00030003179563209414,
  1.549907202580414e-09,
  1.0717731591114443e-08,
  8.066617010626942e-05,
  0.002356284996494651,
  1.601079748070333e-05,
  6.852798861700649e-08,
  0.00041414296720176935,
  0.0007948220008984208,
  0.015468575991690159,
  0.00026569075998850167,
  0.00016894099826458842,
  1.12097595774685

In [107]:
data

,text,book_name,author,author_num
0,"During this time, clouds passed above their he...",Les_Misérables,Victor Hugo,21
1,"But in the hands of Edward Hyde, they soon beg...",The_Strange_Case_of_Dr._Jekyll_and_Mr._Hyde,Robert Louis Stevenson,18
2,Something was written on the flyleaf—just a fe...,A_Little_Princess,Frances Hodgson Burnett,7
3,"If she had been laid low in the streets, in an...",A_Tale_of_Two_Cities,Dickens,4
4,"Not a muscle of her stirred--the hard, defiant...",Woman_in_White,Wilkie Collins,22
...,...,...,...,...
685,"Up here were only dark, bare rock slopes and t...",lovecraft,H.P. Lovecraft,9
686,"""Oh, Harry, we were sure you were going to -- ...",Harry_Potter_and_the_Sorcerer's_Stone,J.K. Rowling,11
687,"I han’t seen a way to get you safe, and I’ve l...",Great_Expectations,Dickens,4
688,"Some, in the true spirit of reckless enterpris...",The_Last_Man,Shelley,19


In [114]:
gru_pred = np.array(gru_pred)
gru_pred.shape

(690, 23)

In [147]:
accuracy_score(data['author_num'].map(author_label),np.argmax(gru_pred,axis = 1))

0.6521739130434783

In [116]:
data['author_num']

0      21
1      18
2       7
3       4
4      22
       ..
685     9
686    11
687     4
688    19
689    20
Name: author_num, Length: 690, dtype: int64

In [117]:
np.argmax(gru_pred,axis = 1)

array([16, 22, 10, 20,  0,  4, 22,  6, 12, 12, 13, 21, 10, 15,  7,  0, 18,
       22, 22, 13, 15, 14, 10, 20, 16,  1, 22, 21, 15,  4, 11, 12, 14, 16,
       16, 10, 22,  0,  4,  8, 22, 15,  5, 22,  1, 17,  7, 21, 21,  8,  8,
        9, 12,  1,  7,  8, 15, 22, 12, 13, 21, 22,  6, 15, 12,  2,  8,  9,
       16,  1,  8, 11,  4, 19, 22, 22,  9,  6,  8, 11, 21, 15,  1, 12,  5,
        9, 15, 21, 16, 16, 11,  6,  1, 16, 22, 15, 19,  6, 12, 15, 19, 15,
       14, 20,  5, 19, 22, 15, 22, 12,  1, 21,  5, 22, 12, 22, 16,  8, 21,
        9, 20, 13, 22, 11, 11, 15, 22,  6, 21,  5, 12,  1, 13, 22, 21,  7,
       21, 22,  8,  6,  1, 16, 14, 11, 22,  0, 18,  1, 10,  7,  1, 15,  7,
       22, 22, 12, 17,  5, 12,  8,  8,  7, 22, 17, 12, 20,  1,  1, 10,  5,
       19,  6, 19,  8,  5, 22, 12, 22, 22, 21,  8, 19,  7, 16,  4,  5, 13,
       16, 18, 14,  2,  8, 12,  1, 22,  5, 22, 13, 21, 18, 16, 22, 17,  0,
       22, 21, 22, 20,  0, 22, 21, 14, 16,  1,  8, 10, 18, 10,  4, 13, 11,
       15,  5, 15, 15, 22

In [118]:
data

,text,book_name,author,author_num
0,"During this time, clouds passed above their he...",Les_Misérables,Victor Hugo,21
1,"But in the hands of Edward Hyde, they soon beg...",The_Strange_Case_of_Dr._Jekyll_and_Mr._Hyde,Robert Louis Stevenson,18
2,Something was written on the flyleaf—just a fe...,A_Little_Princess,Frances Hodgson Burnett,7
3,"If she had been laid low in the streets, in an...",A_Tale_of_Two_Cities,Dickens,4
4,"Not a muscle of her stirred--the hard, defiant...",Woman_in_White,Wilkie Collins,22
...,...,...,...,...
685,"Up here were only dark, bare rock slopes and t...",lovecraft,H.P. Lovecraft,9
686,"""Oh, Harry, we were sure you were going to -- ...",Harry_Potter_and_the_Sorcerer's_Stone,J.K. Rowling,11
687,"I han’t seen a way to get you safe, and I’ve l...",Great_Expectations,Dickens,4
688,"Some, in the true spirit of reckless enterpris...",The_Last_Man,Shelley,19


In [127]:
text = '''
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. The sciences, each straining in its own direction, have hitherto harmed us little; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the revelation or flee from the deadly light into the peace and safety of a new dark age.

Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and human race form transient incidents. They have hinted at strange survivals in terms which would freeze the blood if not masked by a bland optimism. But it is not from them that there came the single glimpse of forbidden eons which chills me when I think of it and maddens me when I dream of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together of separated things—in this case an old newspaper item and the notes of a dead professor. I hope that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part he knew, and that he would have destroyed his notes had not sudden death seized him.

My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intensified by the obscurity of the cause of death. The professor had been stricken whilst returning from the Newport boat; falling suddenly, as witnesses said, after having been jostled by a nautical-looking negro who had come from one of the queer dark courts on the precipitous hillside which formed a short cut from the waterfront to the deceased's home in Williams Street. Physicians were unable to find any visible disorder, but concluded after perplexed debate that some obscure lesion of the heart, induced by the brisk ascent of so steep a hill by so elderly a man, was responsible for the end. At the time I saw no reason to dissent from this dictum, but latterly I am inclined to wonder—and more than wonder.

As my granduncle's heir and executor, for he died a childless widower, I was expected to go over his papers with some thoroughness; and for that purpose moved his entire set of files and boxes to my quarters in Boston. Much of the material which I correlated will be later published by the American Archeological Society, but there was one box which I found exceedingly puzzling, and which I felt much averse from showing to other eyes. It had been locked, and I did not find the key till it occurred to me to examine the personal ring which the professor carried always in his pocket. Then, indeed, I succeeded in opening it, but when I did so seemed only to be confronted by a greater and more closely locked barrier. For what could be the meaning of the queer clay bas-relief and the disjointed jottings, ramblings, and cuttings which I found? Had my uncle, in his latter years, become credulous of the most superficial impostures? I resolved to search out the eccentric sculptor responsible for this apparent disturbance of an old man's peace of mind.
'''

In [121]:
author_dictionary = {0: 'Agatha Christie',
 1: 'Arthur Conan Doyle',
 2: 'Austen',
 3: 'Daniel Defoe',
 4: 'Dickens',
 5: 'Emily Bronte',
 6: 'Fitzgerald',
 7: 'Frances Hodgson Burnett',
 8: 'Gaston Leroux',
 9: 'H.P. Lovecraft',
 10: 'Herman Melville',
 11: 'J.K. Rowling',
 12: 'Jules Verne',
 13: 'Leo Tolstoy',
 14: 'Louisa May Alcott',
 15: 'Mark Twain',
 16: 'Mitchell',
 17: 'Montgomery',
 18: 'Robert Louis Stevenson',
 19: 'Shelley',
 20: 'Stoker',
 21: 'Victor Hugo',
 22: 'Wilkie Collins'}

In [130]:
 i= 0 
 transformer_pred = []
 for _,(para, index, author,author_num) in data.iterrows():
        print(i)
        i+= 1
        _, transformer_output = predict_text(para, transformer_model, tokenizer, device)
        transformer_pred.append(transformer_output)

accuracy_score(data['author_num'],np.argmax(np.array(transformer_pred),axis = 1))

0
[1.04910405e-05 9.96407092e-01 1.25793784e-04 1.16460003e-04
 2.24851057e-04 4.76795431e-05 6.42021711e-04 7.17527582e-05
 1.33246271e-04 2.75707222e-04 1.20679710e-04 4.80651433e-05
 6.04923407e-05 8.10142592e-05 6.74591138e-05 1.52862194e-04
 2.89098854e-04 2.75223192e-05 4.23815982e-05 3.45170381e-04
 2.81907298e-04 3.37103585e-04 9.11587413e-05]
1
[5.76744787e-05 8.54061145e-05 3.48711881e-04 6.19409839e-05
 1.08221044e-04 3.05577560e-04 2.37135027e-05 8.51343357e-05
 7.91414786e-05 2.60262255e-04 1.21420715e-04 1.17027128e-04
 6.93524489e-04 1.73391047e-04 5.69910218e-04 7.27817707e-04
 4.19914024e-04 2.23516501e-04 9.94525313e-01 2.20778620e-05
 7.19598087e-04 1.13268332e-04 1.57388335e-04]
2
[8.9803514e-05 9.5183612e-05 6.8589114e-05 1.0903909e-04 1.6428608e-05
 1.2269302e-04 5.3726351e-05 8.1933606e-05 2.3792716e-04 1.8563532e-04
 9.9769372e-01 2.3736035e-04 1.3166296e-05 3.6397611e-04 5.3579246e-05
 6.3548148e-05 1.2252411e-05 2.3654006e-04 7.1988412e-05 9.9629351e-06
 5.241

0.08405797101449275

In [131]:
np.argmax(np.array(transformer_pred),axis = 1)

array([ 1, 18, 10, 20,  0,  4, 15,  6, 12, 12, 13, 21, 10,  3,  7,  0, 17,
        1, 22,  3,  9, 14, 10, 20, 15,  1, 11, 21,  3,  4, 11,  0, 14, 14,
       14, 10, 22,  0,  4,  8, 22, 12,  5, 16, 17, 17,  7, 21, 21,  8,  8,
       18, 10,  1,  7,  8, 18, 22,  4, 13, 21, 18,  6, 17, 14,  2,  8,  9,
       22,  1,  8, 11,  4, 19, 20, 17,  9,  6,  8,  3, 21,  0,  1, 18,  5,
        9, 20, 21, 14, 16, 11,  6,  1, 16, 22, 15, 19,  6, 19, 15, 19, 13,
       14, 20,  5, 19,  3,  3, 22, 12,  1, 21,  5,  3, 12, 14,  8,  8, 19,
        9, 20, 18, 21, 11, 11, 20, 15,  6, 21,  5, 12,  1, 13, 22, 21,  7,
       21,  3,  8,  6,  1, 16, 14, 11, 18,  0, 10,  9, 10, 10,  1,  2,  7,
       22, 15, 12, 17,  5,  3,  8,  8,  7, 22, 17, 12, 20,  1,  1, 10,  5,
       19, 10, 19,  8, 10,  1, 12, 22, 11, 21,  8, 19,  7,  1,  4, 10, 13,
        3, 18, 20,  2,  8, 12,  1, 13,  5, 22, 13, 21, 20, 16, 18, 17, 20,
        2, 21,  2, 19, 15,  2, 10,  9, 16,  1,  8, 10, 20, 10,  4,  3, 11,
       18,  5, 12, 18,  2

In [134]:
data['author_num'].map(author_label)

0       1
1      18
2      10
3      20
4       0
       ..
685    22
686    11
687    20
688    16
689    13
Name: author_num, Length: 690, dtype: int64

In [133]:
author_label = {22: 0,
 21: 1,
 10: 2,
 15: 3,
 12: 4,
 17: 5,
 2: 6,
 14: 7,
 16: 8,
 13: 9,
 7: 10,
 11: 11,
 1: 12,
 20: 13,
 5: 14,
 3: 15,
 19: 16,
 6: 17,
 18: 18,
 0: 19,
 4: 20,
 8: 21,
 9: 22}

In [267]:
data_with_genre['author']

,text,author,author_num_1,author_num_2
0,Happy families are all alike; every unhappy fa...,Leo Tolstoy,13,9
1,"""Ah, ah, ah! Oo!..."" he muttered, recalling ev...",Leo Tolstoy,13,9
2,But he felt all the difficulty of his position...,Leo Tolstoy,13,9
3,"Stepan Arkadyevitch's eyes asked: ""Why do you ...",Leo Tolstoy,13,9
4,"Beg her forgiveness, sir. There's no help for ...",Leo Tolstoy,13,9
...,...,...,...,...
13149,"He returned after midnight, and, instead of go...",Emily Bronte,5,14
13150,“Strange happiness! If you would hear me witho...,Emily Bronte,5,14
13151,"The following evening was very wet: indeed, it...",Emily Bronte,5,14
13152,"Hareton, with a streaming face, dug green sods...",Emily Bronte,5,14


In [136]:
accuracy_score(data['author_num'].map(author_label),np.argmax(np.array(gru_pred),axis = 1))

0.6521739130434783

In [141]:
i=0
cnn_pred = []
for _,(para, index, author,author_num) in data.iterrows():
        print(i)
        i+= 1
        cnn_output = majority_vote(models, para)
        cnn_pred.append(cnn_output)


0
1/1 [==============================] - 0s 86ms/step
1
1/1 [==============================] - 0s 71ms/step
2
1/1 [==============================] - 0s 74ms/step
3
1/1 [==============================] - 0s 79ms/step
4
1/1 [==============================] - 0s 90ms/step
5
1/1 [==============================] - 0s 82ms/step
6
1/1 [==============================] - 0s 75ms/step
7
1/1 [==============================] - 0s 80ms/step
8
1/1 [==============================] - 0s 87ms/step
9
1/1 [==============================] - 0s 75ms/step
10
1/1 [==============================] - 0s 86ms/step
11
1/1 [==============================] - 0s 70ms/step
12
1/1 [==============================] - 0s 75ms/step
13
1/1 [==============================] - 0s 80ms/step
14
1/1 [==============================] - 0s 71ms/step
15
1/1 [==============================] - 0s 70ms/step
16
1/1 [==============================] - 0s 73ms/step
17
1/1 [==============================] - 0s 75ms/step
18
1/1 [============

0.08405797101449275

In [143]:
accuracy_score(data['author_num'].map(author_label),np.argmax(np.array(transformer_pred),axis = 1))

0.9855072463768116

In [146]:
data

,text,book_name,author,author_num
0,"During this time, clouds passed above their he...",Les_Misérables,Victor Hugo,21
1,"But in the hands of Edward Hyde, they soon beg...",The_Strange_Case_of_Dr._Jekyll_and_Mr._Hyde,Robert Louis Stevenson,18
2,Something was written on the flyleaf—just a fe...,A_Little_Princess,Frances Hodgson Burnett,7
3,"If she had been laid low in the streets, in an...",A_Tale_of_Two_Cities,Dickens,4
4,"Not a muscle of her stirred--the hard, defiant...",Woman_in_White,Wilkie Collins,22
...,...,...,...,...
685,"Up here were only dark, bare rock slopes and t...",lovecraft,H.P. Lovecraft,9
686,"""Oh, Harry, we were sure you were going to -- ...",Harry_Potter_and_the_Sorcerer's_Stone,J.K. Rowling,11
687,"I han’t seen a way to get you safe, and I’ve l...",Great_Expectations,Dickens,4
688,"Some, in the true spirit of reckless enterpris...",The_Last_Man,Shelley,19
